# OpenCV Face Detection Webcam

In this notebook, opencv face detection will be applied to webcam images.

To run all cells in this notebook a webcam and DisplayPort output monitor are required.  

References:

https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml

### Step 1: Load the overlay

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

### Step 2: Initialize Webcam and DisplayPort Out

In [2]:
# monitor configuration: 1280*720 @ 60Hz
displayport = DisplayPort()
displayport.configure(VideoMode(1280, 720, 24), PIXEL_RGB)

Ignoring unconnected port


ValueError: mode: VideoMode: width=1280 height=720 bpp=24 fps=60 is not supported

In [ ]:
# monitor (output) frame buffer size
frame_out_w = 1280
frame_out_h = 720
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [ ]:
# initialize camera from OpenCV
import cv2

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

### Step 3: Show input frame on DisplayPort output

In [ ]:
# Capture webcam image
import numpy as np

ret, frame_vga = videoIn.read()

# Display webcam image via DisplayPort Out
if (ret):          
    outframe = displayport.newframe()
    outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]    
    displayport.writeframe(outframe)
else:
    raise RuntimeError("Failed to read from camera.")

### Step 4: Now use matplotlib to show image inside notebook

In [ ]:
# Output webcam image as JPEG
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
plt.imshow(frame_vga[:,:,[2,1,0]])
plt.show()

### Step 5: Apply the face detection to the input

In [ ]:
import cv2

np_frame = frame_vga

face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_eye.xml')

gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x,y,w,h) in faces:
    cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = np_frame[y:y+h, x:x+w]

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

### Step 6: Show results on DisplayPort output

In [ ]:
# Output OpenCV results via DisplayPort
outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
displayport.writeframe(outframe)

### Step 7: Now use matplotlib to show image inside notebook

In [ ]:
# Output OpenCV results via matplotlib
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
plt.imshow(np_frame[:,:,[2,1,0]])
plt.show()

### Step 8: Release camera and DisplayPort

In [ ]:
videoIn.release()
displayport.close()

Copyright (c) 2022 Xilinx, Inc 
<br>
Copyright (C) 2022-2025 Advanced Micro Devices, Inc. 
<br>
SPDX-License-Identifier: BSD-3-Clause 